## Использование api и обращение к серверу.

Реализация rest api на базе flask 

- dataset: https://www.kaggle.com/vikasukani/diabetes-data-set
- Данные о предсказании наличия диабета у пациентов. 
- model: AdaBoostClassifier

Как использовать:
1. С помощью Model_creation.ipynb обучить модель на данных diabetes-dataset.csv и сохранить в ada_model.dill
2. Используя run_server.py запустить сервер в терминале командой "python run_server.py" либо через IDE.
3. Запустив сервер, с помощью Prediction.ipynb можно обращаться к api

In [32]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [33]:
import urllib.request
import json      

def get_prediction(pregnancies, glucose, bloodpressure, skinthickness, insulin, bmi, diabetespedigreefunction, age):
    body = {"pregnancies": pregnancies,
            "glucose": glucose,
            "bloodpressure": bloodpressure,
            "skinthickness": skinthickness,
            "insulin": insulin,
            "bmi": bmi,
            "diabetespedigreefunction": diabetespedigreefunction,
            "age": age}
    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

POST запрос к серверу:

In [35]:
get_prediction(2, 122, 70, 27, 0, 36.80, 0.34, 27)

0.4995592785649417

Если обращение некорректное, то получим сообщение с ошибкой, отловенное в скрипте run_server.py:

In [37]:
get_prediction("f", 122, 70, 27, 0, 36.80, 0.34, 27)

"could not convert string to float: 'f'"

Финальное предсказание на тестовых данных:

In [39]:
%%time
predictions = X_test.apply(lambda x: get_prediction(x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7]), 1)

CPU times: user 256 ms, sys: 65.6 ms, total: 321 ms
Wall time: 55.1 s


In [41]:
roc_auc_score(y_score=predictions.values, y_true=y_test)

0.9265688329839273

Неплохой ROC AUC Score, модель и запросы отработали корректно